# Final Project for the University Course: AI in Physics

Created by: **Jonah Fischer**

This notebook uses the higgs dataset from: https://archive.ics.uci.edu/dataset/280/higgs to distinguish between a signal process which produces Higgs bosons and a background process which does not.

In [ ]:
!wget https://archive.ics.uci.edu/static/public/280/higgs.zip

--2024-01-10 13:22:01--  https://archive.ics.uci.edu/static/public/280/higgs.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘higgs.zip’

higgs.zip               [     <=>            ]   1.60G  58.3MB/s               

In [ ]:
!unzip higgs.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import torch
from torch import nn
from sklearn.model_selection import train_test_split

In [ ]:
# Read data
df = pd.read_csv('/content/HIGGS.csv.gz', nrows=3000000)
df.head()

In [ ]:
# Change labels
df.columns = ['label', 'f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27']
df.head()

In [ ]:
# Split data
X = df.drop(['label'], axis=1)
y = df['label']
X.head()

In [ ]:
y.head()

In [ ]:
df.info()

In [ ]:
# Visualize feature distributions
sns.set_theme(style="darkgrid")

cols = 4
fig, axes = plt.subplots(ncols=cols, nrows=7, sharey=False, figsize=(14,14), constrained_layout=True)

for i in range(28):
    feature = f'f{i}'
    col = i % cols
    row = i // cols
    sns.histplot(df[feature], ax=axes[row][col], kde=True)

In [ ]:
# cor = df.corr()

# plt.figure(figsize=(10, 6))
# sns.heatmap(cor, annot=True)

In [ ]:
# Count signal and background data
value_counts = df['label'].value_counts()
sns.barplot(x=value_counts.index, y=value_counts.values)
plt.xlabel('Value')
plt.ylabel('Count')

In [ ]:
# Feature selection
#X = X.drop(['f21', 'f22', 'f23', 'f24', 'f25', 'f26'], axis=1)
#X.info()

In [ ]:
# Turn data into tensor
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))
# del df

In [ ]:
# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Setup device agnostic code
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

In [ ]:
# Build a binary classification model
class Classifier(nn.Module):
  def __init__(self, input_features, output_features, hidden_units=8):
    '''Initializes classification model.

    Args:
      input_features (int): Number of input features to the model
      output_features (int): Number of output features (number of output classes)
      hidden_units (int): Number of hidden units between the layers, default 8
    '''
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
        nn.Linear(in_features=input_features, out_features=hidden_units),
        nn.Tanh(),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.Tanh(),
        nn.Linear(in_features=hidden_units, out_features=output_features)
    )

  def forward(self, x):
    return self.linear_layer_stack(x)

# Create an instance of the model and send it to the target device
model = Classifier(input_features=X.shape[1],
                    output_features=1,
                    hidden_units=128).to(device)

In [ ]:
model

In [ ]:
# Calculate accuracy - out of a 100 examples, what percentage does our model get right?
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

In [ ]:
# Create a loss function
loss_fn =  nn.BCEWithLogitsLoss()

# Create an optimizer
optimizer = torch.optim.Adam(params=model.parameters(),
                            lr=0.01)

torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set the number of epoch
epochs = 1500

# Put the data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(epochs):
  ### Training
  model.train()

  # 1. Forward pass
  y_logits = model(X_train.type(torch.float)).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred_probs -> pred labels

  # 2. Calculate the loss/accuracy
  loss = loss_fn(y_logits, # nn.BCEWithLogitsLoss expects raw logits as input
                 y_train)
  acc = accuracy_fn(y_true=y_train,
                    y_pred=y_pred)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. Loss backward (backpropagation)
  loss.backward()

  #5. Optimizer step (gradient descent)
  optimizer.step()

  ### Testing
  model.eval()
  with torch.inference_mode():
    # 1. Forward pass
    test_logits = model(X_test.type(torch.float)).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))

    # 2. Calculate test loss/acc
    test_loss = loss_fn(test_logits,
                        y_test)
    test_acc = accuracy_fn(y_true=y_test,
                           y_pred=test_pred)

  # Print out what's happenin'
  if epoch % 100 == 0:
    print(f'Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%')

In [ ]:
# Evaluate the model's performace
from sklearn.metrics import precision_score, confusion_matrix, f1_score

y_preds = model(X_test.type(torch.float)).squeeze()
y_preds = torch.round(torch.sigmoid(y_preds)).cpu().detach().numpy()

precision = precision_score(y_test.cpu().numpy(), y_preds)
c_mat = confusion_matrix(y_test.cpu().numpy(), y_preds, normalize='true')
F1 = f1_score(y_test.cpu().numpy(), y_preds)

print(f'Precision: {precision}')
print(f'Confusion Matrix: \n{c_mat}')
print(f'F1-Score: {F1}')

In [ ]:
X_train_v1 , X_test_v1, y_train_v1, y_test_v1 = X_train.clone(), X_test.clone(), y_train.clone(), y_test.clone()
model_1 = Classifier(input_features=28,
                    output_features=1,
                    hidden_units=128).to(device)

In [ ]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import mutual_info_classif

#f1_score_list = []

#for k in range(1, 10):
#  selector = SelectKBest(mutual_info_classif, k=k)
#  selector.fit(X_train_v1.cpu().numpy(), y_train_v1.cpu().numpy())

#  sel_X_train_v1 = selector.transform(X_train_v1)
#  sel_X_test_v1 = selector.transform(X_test_v1)

#  model_1.fit(sel_X_train_v1, y_train_v1)
#  kbest_preds = model_1(sel_X_test_v1)

#  f1_score_kbest = round(f1_score(y_test_v1, kbest_preds, average='weighted'), 3)

#  f1_score_list.append(f1_score_kbest)

In [ ]:
#fig, ax = plt.subplots()

#x = np.arange(1, 29)
#y = f1_score_list

#sns.barplot(x, y)
#ax.set_xlabel('Number of selected features')
#ax.set_ylabel('F1-score (weighted)')
#ax.set_xticks(x)
#ax.set_xtickslabels(x, fontsize=12)

#for i, v in enumerate(y):
#  plt.text(x=i+1, y=v+0.05, s=str(v), ha='center')

#plt.tight_layout()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_X_train_v1 = scaler.fit_transform(X_train_v1.cpu().numpy())

fig, ax = plt.subplots(figsize=(12,9), constrained_layout=True)

x = X.columns
y = scaled_X_train_v1.var(axis=0)

ax.bar(x, y, width=0.2)
ax.set_xlabel('Features')
ax.set_ylabel('Variance')

for i, v in enumerate(y):
  plt.text(x=i, y=v+0.002, s=str(round(v, 3)), ha='center')

fig.autofmt_xdate()